# Regression Model B: Field 2

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('mice')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("mice"):
“installation of package ‘mice’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(mice)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 2 (F2)

Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E2" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E2" ),]
dE4=d4[which(d4$Estate=="E2" ),]
dE5=d5[which(d5$Estate=="E2" ),]

## Variables

Declare the variables. The variables in which pruning time, pruning cost, manuring time, manusring cost, P&D time and P&D cost contains zero values. In this case, we assume that the data is not available (NA). Then, we construct a new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL

foliar=d2$Diff

prun.time=dE3$Frequency
prun.time[prun.time == 0] <- NA

prun.cost=dE3$PruningCost
prun.cost[prun.cost == 0] <- NA

manuring.time=dE4$Frequency
manuring.time[manuring.time == 0] <- NA

manuring.cost=dE4$ManuringCost
manuring.cost[manuring.cost == 0] <- NA

pd.time=dE5$Frequency
pd.time[pd.time == 0] <- NA

pd.cost=dE5$PDCost
pd.cost[pd.cost == 0] <- NA


df1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Identify missing values in each variables

In this case, we impute the missing values. Imputation is the process of replacing missing data with substituted values.  

This study use MICE (Multivariate Imputation via Chained Equations). MICE (Multivariate Imputation via Chained Equations) is one of the commonly used package by R users. Creating multiple imputations as compared to a single imputation (such as mean) takes care of uncertainty in missing values.

MICE assumes that the missing data are Missing at Random (MAR), which means that the probability that a value is missing depends only on observed value and can be predicted using them. It imputes data on a variable by variable basis by specifying an imputation model per variable.


The mice package has function known as md.pattern(). It returns a tabular for of missing value present in each variable in a data set.

In [6]:
md.pattern(df1)

,year,month,yield,rainfall,foliar,pd.time,pd.cost,manuring.time,manuring.cost,prun.time,prun.cost,
13,1,1,1,1,1,1,1,1,1,1,1,0
15,1,1,1,1,1,1,1,1,1,0,0,2
4,1,1,1,1,1,1,1,0,0,1,1,2
1,1,1,1,1,1,0,0,1,1,1,1,2
7,1,1,1,1,1,1,1,0,0,0,0,4
3,1,1,1,1,1,0,0,1,1,0,0,4
1,1,1,1,1,1,0,0,0,0,1,1,4
,0,0,0,0,0,5,5,12,12,25,25,84


## Imputation

Here is an explanation of the parameters used:  
m  – Refers to 5 imputed data sets.  
maxit – Refers to no. of iterations taken to impute missing values.  
method – Refers to method used in imputation. We used predictive mean matching.

In [7]:
imputed_Data <- mice(df1[,-1:-4], m=5, maxit = 50, method = 'pmm', seed = 500)
summary(imputed_Data)


 iter imp variable
  1   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  1   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   3  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   4  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  2   5  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   1  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   2  prun.time  prun.cost  manuring.time  manuring.cost  pd.time  pd.cost
  3   3  prun.time  prun.cost  manuring.time

Multiply imputed data set
Call:
mice(data = df1[, -1:-4], m = 5, method = "pmm", maxit = 50, 
    seed = 500)
Number of multiple imputations:  5
Missing cells per column:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
            0            25            25            12            12 
      pd.time       pd.cost 
            5             5 
Imputation methods:
       foliar     prun.time     prun.cost manuring.time manuring.cost 
        "pmm"         "pmm"         "pmm"         "pmm"         "pmm" 
      pd.time       pd.cost 
        "pmm"         "pmm" 
VisitSequence:
    prun.time     prun.cost manuring.time manuring.cost       pd.time 
            2             3             4             5             6 
      pd.cost 
            7 
PredictorMatrix:
              foliar prun.time prun.cost manuring.time manuring.cost pd.time
foliar             0         0         0             0             0       0
prun.time          1         0         1           

Since there are 5 imputed data sets, we select any using complete() function.
Get complete data (2nd out of 5)

Get the complete data after impute the missing value.

In [8]:
completeData <- complete(imputed_Data,2)
df2=data.frame(df1[,1:4],completeData)

## Correlation

Check the correlations with all variables. 

In [9]:
r1=cor(df2[,-1:-2])

corrplot(r1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(r1, type="upper",col=brewer.pal(n=8, name="RdYlBu"))


## Regression

Fit the data using multiple regression for all variables. 

In [10]:
k1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=df2)
summary(k1)
plot(k1,which=1)
plot(k1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = df2)

Residuals:
     Min       1Q   Median       3Q      Max 
-123.413  -28.634   -6.076   23.669  108.384 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)  
(Intercept)   78.9635549 38.3767971   2.058   0.0471 *
rainfall       0.0222467  0.0351773   0.632   0.5312  
foliar        -4.2162035  2.4291680  -1.736   0.0914 .
prun.time     -0.3403817  6.5043623  -0.052   0.9586  
prun.cost      0.0078540  0.0085985   0.913   0.3673  
manuring.time 25.2935944 18.6764588   1.354   0.1843  
manuring.cost -0.0002909  0.0007526  -0.386   0.7015  
pd.time       18.4376220 12.8093691   1.439   0.1589  
pd.cost       -0.0070979  0.0083042  -0.855   0.3985  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 48.1 on 35 degrees of freedom
Multiple R-squared:  0.1877,	Adjusted R-squared:  0.002055 
F-statis

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 

The stepwise method incorporate with VIF factor is better than anova method in order to select the significant variables. 

### Using ANOVA

In [11]:
anova(k1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,1168.28697,1168.28697,0.50491501,0.4820568
foliar,1,5474.09107,5474.09107,2.36581492,0.1330117
prun.time,1,34.79227,34.79227,0.01503667,0.9031062
prun.cost,1,1244.70470,1244.70470,0.53794153,0.4681717
manuring.time,1,5408.03662,5408.03662,2.33726724,0.1352993
manuring.cost,1,591.65110,591.65110,0.25570217,0.6162583
pd.time,1,3103.55391,3103.55391,1.34130654,0.2546457
pd.cost,1,1690.41903,1690.41903,0.73057216,0.3985108
Residuals,35,80984.01360,2313.82896,NA,NA


### Using stepwise

In [12]:
selectedMod <- step(k1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=348.78
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- prun.time      1       6.3 80990 346.79
- manuring.cost  1     345.6 81330 346.97
- rainfall       1     925.4 81909 347.28
- pd.cost        1    1690.4 82674 347.69
- prun.cost      1    1930.5 82914 347.82
<none>                       80984 348.78
- manuring.time  1    4243.9 85228 349.03
- pd.time        1    4793.9 85778 349.31
- foliar         1    6970.4 87954 350.42

Step:  AIC=346.79
yield ~ rainfall + foliar + prun.cost + manuring.time + manuring.cost + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.cost  1     340.1 81330 344.97
- rainfall       1     919.6 81910 345.28
- pd.cost        1    1691.8 82682 345.70
- prun.cost      1    1924.2 82915 345.82
<none>                       80990 346.79
- manuring.time  1    4443.1 85433 347.14
- pd.time        1    4790.7 85781 347.32
- 


Call:
lm(formula = yield ~ foliar + manuring.time, data = df2)

Residuals:
     Min       1Q   Median       3Q      Max 
-116.841  -31.547   -5.477   29.811   99.455 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    115.779     23.430   4.941 1.35e-05 ***
foliar          -3.254      2.097  -1.552    0.128    
manuring.time   25.382     15.825   1.604    0.116    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 46.69 on 41 degrees of freedom
Multiple R-squared:  0.1036,	Adjusted R-squared:  0.05989 
F-statistic:  2.37 on 2 and 41 DF,  p-value: 0.1062


       foliar manuring.time 
     1.002614      1.002614 



Call:
lm(formula = yield ~ foliar + manuring.time, data = df2)

Residuals:
     Min       1Q   Median       3Q      Max 
-116.841  -31.547   -5.477   29.811   99.455 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    115.779     23.430   4.941 1.35e-05 ***
foliar          -3.254      2.097  -1.552    0.128    
manuring.time   25.382     15.825   1.604    0.116    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 46.69 on 41 degrees of freedom
Multiple R-squared:  0.1036,	Adjusted R-squared:  0.05989 
F-statistic:  2.37 on 2 and 41 DF,  p-value: 0.1062


## Prediction

Predict with the resample values.

In [13]:
set.seed(123)
df3=sample(1:nrow(df2),12)
df4=df2[df3,]
estimate=predict(selectedMod,newdata=df4)
estimate
sum.estimate=sum(estimate)
sum.estimate

df5=data.frame(estimate,sum.estimate)

13       34       18       37       38        2       21       43 
141.1617 111.8796 124.8939 166.5442 137.9081 137.9081 115.1332 147.0228 
      20       16       33       15 
118.3867 131.4010 140.5156 160.0370

[1] 1632.792